# 圖片自動化後製處理
1.設定輸出照片大小與來源資料位置<br>
2.始以「照片中的非(255,255,255)之位置」判斷商品主體的位置，並擷取該影像<br>
3.判斷擷取影像的長短邊<br>
4.依照設定的邊框像素大小補足長邊<br>
5.將剩餘短邊以白底背景填滿成#1輸入的正方形大小

In [22]:
import cv2
import os
import glob
import numpy as np

#設定輸出照片大小及資料來源
size = int(input("輸出照片大小(正方形)：(Ex:200)"))
root_path = input("來源資料夾路徑：").encode('utf8') 
img_list = os.listdir(root_path.decode('utf8'))

輸出照片大小(正方形)：(Ex:200)1000
來源資料夾路徑：C:\Users\晴\Desktop\研究所\others\try


In [23]:
#建立儲存路徑
save_path = input("儲存資料夾路徑：")
if not os.path.exists(save_path):
    os.mkdir(save_path)
    print('make direction')
save_path = save_path.encode('utf8')

#設定留邊像素
x = int(size * .15) 

儲存資料夾路徑：C:\Users\晴\Desktop\研究所\others\tryre
make direction


In [24]:
from tqdm.notebook import tqdm
for i in tqdm(range(len(img_list))):
    #判斷影像序列中是不是全部都是影像，如果不是，跳過
    if not img_list[i].split('.')[-1] == 'jpg' or img_list[i].split('.')[-1] == 'png': 
        print(str(img_list[i]) + ' is not a .jpg/.png file.')
        continue
        
    full_path = os.path.join(root_path.decode('utf8'), img_list[i])
    now_img = cv2.imdecode(np.fromfile(full_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    
    #找到X軸與Y軸最大跟最小哪邊不是白色的地方
    Ymin = np.min(np.where(now_img!=255)[0])
    Ymax = np.max(np.where(now_img!=255)[0])
    Xmin = np.min(np.where(now_img!=255)[1])
    Xmax = np.max(np.where(now_img!=255)[1])
       
    cropped_img = now_img[Ymin:Ymax, Xmin:Xmax]
    #cv2.copyMakeBoreder(輸入影像, 上, 下, 左, 右, 模式, 填充影像像素)
    cropped_img = cv2.copyMakeBorder(
        cropped_img, x , x , x , x , cv2.BORDER_CONSTANT, value=(255, 255, 255)
    )
    

978833.png is not a .jpg/.png file.
978835.png is not a .jpg/.png file.
978841.png is not a .jpg/.png file.
979000.png is not a .jpg/.png file.



In [25]:

    height, width, _ = cropped_img.shape
    
    #先判斷影像是哪一邊長，判斷完後針對長邊與短邊的差除以二，再針對短邊做補償
    if height > width: 
        #因為使用整除2的關係，如果被除數是1的話，除完之後compensate還會是1，故會有一邊多出1個pixel，影像就不是正方形
        compensate = (height - width) // 2 
        cropped_img = cv2.copyMakeBorder(
            cropped_img, 0, 0, compensate, compensate, cv2.BORDER_CONSTANT, value=(255, 255, 255)
        )
    elif height < width:
        compensate = (width - height) // 2
        cropped_img = cv2.copyMakeBorder(
            cropped_img, compensate, compensate, 0, 0, cv2.BORDER_CONSTANT, value=(255, 255, 255)
        )

    #如果影像不是正方形的，針對短邊補償1個pixel
    if cropped_img.shape[0] > cropped_img.shape[1]: 
        cropped_img = cv2.copyMakeBorder(
            cropped_img, 0, 0, 1, 0, cv2.BORDER_CONSTANT, value=(255, 255, 255)
        )
    elif cropped_img.shape[0] < cropped_img.shape[1]:
        cropped_img = cv2.copyMakeBorder(
            cropped_img, 1, 0, 0, 0, cv2.BORDER_CONSTANT, value=(255, 255, 255)
        )
        
    #把影像Resize成某個大小
    cropped_img = cv2.resize(cropped_img, (size, size))
        
    cv2.imencode('.jpg', cropped_img)[1].tofile(os.path.join(save_path.decode('utf8'), img_list[i]))